# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups?
            * If so, how do you plan to account for this in your experimental design?

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

df = pd.read_csv('homepage_actions.csv')

In [31]:
df

,timestamp,id,group,action
0,42:27.8,804196,experiment,view
1,19:03.5,434745,experiment,view
2,36:00.9,507599,experiment,view
3,59:02.6,671993,control,view
4,26:14.5,536734,experiment,view
...,...,...,...,...
8183,11:42.0,192060,experiment,view
8184,42:12.8,755912,experiment,view
8185,01:09.0,458115,experiment,view
8186,08:51.6,505451,control,view


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [33]:
df['id'].nunique()

6328

In [34]:
df['action'].value_counts()

,count
action,
view,6328
click,1860


In [35]:
df['group'].value_counts()

,count
group,
control,4264
experiment,3924


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [36]:
"""

Null Hypothesis: The experimental homepage had the same effectiveness or less as the control group homepage.
Alternative Hypothesis: The experimental homepage was more effective than that of the control group.

Conduct a 2-sample one-tailed t-test to test the hypothesis.
H1: u(cont) < u(exp)
H0: u(cont) >= u(exp)

Use a significance level of 0.05.

"""

'\n\nNull Hypothesis: The experimental homepage had the same effectiveness or less as the control group homepage.\nAlternative Hypothesis: The experimental homepage was more effective than that of the control group.\n\nConduct a 2-sample one-tailed t-test to test the hypothesis.\nH1: u(cont) < u(exp)\nH0: u(cont) >= u(exp)\n\n\n'

In [37]:
df['success'] = df['action'].replace({'click': 1, 'view': 0})
df

<ipython-input-37-6cf55c717e4e>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['success'] = df['action'].replace({'click': 1, 'view': 0})


,timestamp,id,group,action,success
0,42:27.8,804196,experiment,view,0
1,19:03.5,434745,experiment,view,0
2,36:00.9,507599,experiment,view,0
3,59:02.6,671993,control,view,0
4,26:14.5,536734,experiment,view,0
...,...,...,...,...,...
8183,11:42.0,192060,experiment,view,0
8184,42:12.8,755912,experiment,view,0
8185,01:09.0,458115,experiment,view,0
8186,08:51.6,505451,control,view,0


In [38]:
control = df[df['group'] == 'control']
experiment = df[df['group'] == 'experiment']

In [39]:
contol_success_count = control['success'].value_counts()
experiment_success_count = experiment['success'].value_counts()

print(contol_success_count, experiment_success_count)

success
0    3332
1     932
Name: count, dtype: int64 success
0    2996
1     928
Name: count, dtype: int64


In [40]:
control_mean = control['success'].mean()
experiment_mean = experiment['success'].mean()
diff_mean = experiment_mean - control_mean

print(diff_mean)

control_var = control['success'].var()
experiment_var = experiment['success'].var()

print(control_mean, experiment_mean)
print(control_var, experiment_var)

0.017919265290041758
0.21857410881801126 0.23649337410805302
0.17083953332902724 0.18061028519697947


In [41]:
pooled_var = ((control_var * (len(control) - 1)) + (experiment_var * (len(experiment) - 1))) / (len(control) + len(experiment) - 2)
pooled_std = np.sqrt(pooled_var)
pooled_std

np.float64(0.4189534562000106)

In [46]:
# went through step by step process of calculating the t-statistic for a two-sample t-test
t_stat_calc = (control_mean - experiment_mean) / (pooled_std * np.sqrt((1 / len(control)) + (1 / len(experiment))))
t_stat_calc

np.float64(-1.9334751824864993)

In [44]:
# check the work above with the stats formula, about 2/1000 of each other
t_stat, p_value = stats.ttest_ind(control['success'], experiment['success'], equal_var=False)
print(t_stat, p_value)

-1.9312441445072492 0.05348777384439879


In [57]:
"""

With a p-value (0.05348) greater than the alpha of 0.05, we cannot reject the null hypothesis.

"""

'\n\nWith a p-value (0.05348) greater than the alpha of 0.05, we cannot reject the null hypothesis.\n\n'

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not.

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group.
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate.
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group.

In [51]:
control_clickrate = control['success'].mean()
exp_expectclicks = control_clickrate * len(experiment)
print(exp_expectclicks)

857.6848030018762


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [63]:
exp_actualclicks = experiment['success'].sum()
control_var = len(control) * control_clickrate * (1 - control_clickrate)
control_std = np.sqrt(control_var)
z_score = (exp_actualclicks - exp_expectclicks) / control_std
print(z_score)


2.6055375912248526


### Step 3:
Finally, calculate a p-value using the normal distribution based on this z-score.

In [64]:
p_value = 1 - stats.norm.cdf(z_score)
print(p_value)

0.004586510360224505


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.